In [1]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [2]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [7]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [41]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    #init.run() # initialize all variables
    sess.run(init)
    result = f.eval()
    print(result)

42


In [9]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [10]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [11]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

x2.graph is graph

True

In [12]:
x2.graph is tf.get_default_graph()

False

In [13]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [15]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


In [2]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


In [5]:
housing.data.shape

(20640, 8)

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [32]:
scaled_housing_data_plus_bias[0:2]

array([[ 1.        ,  2.34476576,  0.98214266,  0.62855945, -0.15375759,
        -0.9744286 , -0.04959654,  1.05254828, -1.32783522],
       [ 1.        ,  2.33223796, -0.60701891,  0.32704136, -0.26333577,
         0.86143887, -0.09251223,  1.04318455, -1.32284391]])

In [14]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1., 1.), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

# Original way
#gradients = 2/m * tf.matmul(tf.transpose(X), error)
#training_op = tf.assign(theta, theta - learning_rate * gradients)

# Use autodiff to compute gradients automatically
#gradients = tf.gradients(mse, [theta])[0]
#training_op = tf.assign(theta, theta - learning_rate * gradients)

# Use an Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch = ", epoch, "MSE = ", mse.eval())
            save_path = saver.save(sess, "C:\Boyuan\MyPython\HandsOnML\\tmp\\my_model.ckpt")
        sess.run(training_op)  #the same as training_op.eval()
        
    best_theta = theta.eval()
    print(best_theta)
    save_path = saver.save(sess, "C:\Boyuan\MyPython\HandsOnML\\tmp\\my_model_final.ckpt")

Epoch =  0 MSE =  4.92846


Epoch =  100 MSE =  0.694995


Epoch =  200 MSE =  0.592247


Epoch =  300 MSE =  0.574755


Epoch =  400 MSE =  0.563695


Epoch =  500 MSE =  0.555321
Epoch = 

 600 MSE =  0.548853
Epoch =  700 MSE =  0.543827


Epoch =  800 MSE =  0.539899
Epoch =  900 MSE =  0.536814


[[ 2.06855249]
 [ 0.70124704]
 [ 0.11370201]
 [ 0.01258701]
 [ 0.06007489]
 [-0.00467434]
 [-0.03616179]
 [-1.05723464]
 [-1.01122832]]


In [22]:
import numpy as np
from sklearn.datasets import fetch_california_housing

tf.reset_default_graph()

housing = fetch_california_housing()
m, n = housing.data.shape

theta = tf.get_variable(shape=[n+1, 1], name="theta")
saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, "C:\Boyuan\MyPython\HandsOnML\\tmp\\my_model_final.ckpt")
    best_theta = theta.eval()
    print(best_theta)


INFO:tensorflow:Restoring parameters from C:\Boyuan\MyPython\HandsOnML\tmp\my_model_final.ckpt


[[ 2.06855249]
 [ 0.70124704]
 [ 0.11370201]
 [ 0.01258701]
 [ 0.06007489]
 [-0.00467434]
 [-0.03616179]
 [-1.05723464]
 [-1.01122832]]


In [5]:
y_train = housing.target.reshape(-1, 1)

In [6]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch*n_batches + batch_index)
    random_index = np.random.choice(m, batch_size, replace=False)  # or use indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[random_index]
    y_batch = y_train[random_index]
    return X_batch, y_batch

In [39]:
n_epochs = 1000
learning_rate = 0.005
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n+1, 1], -1., 1.), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

#gradients = 2/m * tf.matmul(tf.transpose(X), error)
#training_op = tf.assign(theta, theta - learning_rate * gradients)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()


with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        #print("Epoch = ", epoch)
        if epoch % 100 == 0:
            print("Epoch = ", epoch, "MSE = ", mse.eval(feed_dict={X: scaled_housing_data_plus_bias, y: y_train}))
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            #if epoch % 100 == 0 and batch_index % 10 == 0 :
            #    print("Epoch = ", epoch, "batch_index = ", batch_index, "MSE = ", mse.eval(feed_dict={X: X_batch, y: y_batch}))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
    best_theta = theta.eval()
    print(best_theta)



Epoch =  0 batch_index =  70 MSE =  11.5307


Epoch =  100 batch_index =  206 MSE =  0.526935


Epoch =  200 batch_index =  206 MSE =  0.527735


Epoch =  300 batch_index =  206 MSE =  0.526361


Epoch =  400 batch_index =  206 MSE =  0.527272


KeyboardInterrupt: 

In [40]:
n_epochs = 1000
learning_rate = 0.005
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch = ", epoch, "MSE = ", mse.eval(feed_dict={X: scaled_housing_data_plus_bias, y: y_train}))
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

Epoch =  0 MSE =  4.37194


Epoch =  100 MSE =  0.533823


Epoch =  200 MSE =  0.525401


Epoch =  300 MSE =  0.529582


Epoch =  400 MSE =  0.524672


Epoch =  500 MSE =  0.52812


Epoch =  600 MSE =  0.524793


KeyboardInterrupt: 

In [ ]:
# Create some variables.
v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

inc_v1 = v1.assign(v1+1)
dec_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
  sess.run(init_op)
  # Do some work with the model.
  inc_v1.op.run()
  dec_v2.op.run()
  # Save the variables to disk.
  save_path = saver.save(sess, "C:\Boyuan\MyPython\HandsOnML\\tmp\\simple_model.ckpt")
  print("Model saved in file: %s" % save_path)

In [18]:
tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("v1", shape=[3])
v2 = tf.get_variable("v2", shape=[5])

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "C:\Boyuan\MyPython\HandsOnML\\tmp\\simple_model.ckpt")
  print("Model restored.")
  # Check the values of the variables
  print("v1 : %s" % v1.eval())
  print("v2 : %s" % v2.eval())

INFO:tensorflow:Restoring parameters from C:\Boyuan\MyPython\HandsOnML\tmp\simple_model.ckpt


Model restored.
v1 : [ 1.  1.  1.]
v2 : [-1. -1. -1. -1. -1.]


In [8]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "C:\Boyuan\MyPython\HandsOnML\\{}\\run-{}\\".format(root_logdir, now)

n_epochs = 10
learning_rate = 0.005
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n+1, 1], -1., 1.), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())  # or file_writer = tf.summary.FileWriter(logdir, sess.graph) 

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 ==0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
    best_theta = theta.eval()
    print(best_theta)

file_writer.close()

[[ 2.06752419]
 [ 0.86637586]
 [ 0.14600717]
 [-0.27849847]
 [ 0.35490558]
 [ 0.0070019 ]
 [-0.04934338]
 [-0.68228167]
 [-0.65818912]]


In [31]:
tf.get_default_graph()

In [9]:
print(error.op.name)

loss/sub


In [10]:
print(mse.op.name)

loss/mse


In [24]:
tf.reset_default_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [26]:
tf.reset_default_graph()

def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")  #reuse existing variable
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):  #create the variable
    threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [31]:
tf.reset_default_graph()

def relu(X):
    threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, threshold, name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index > 0)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

In [32]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "C:\Boyuan\MyPython\HandsOnML\\{}\\run-{}\\".format(root_logdir, now)

init = tf.global_variables_initializer()

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph()) 

with tf.Session() as sess:
    sess.run(init)
    output_val = output.eval(feed_dict={X: [[1, 2, 3], [4, 5, 6]]})

file_writer.close()
print(output_val)

[[ 11.85008621]
 [ 19.41698456]]


In [38]:
from sklearn import datasets
iris = datasets.load_iris()
iris_data_X = iris["data"][:]
iris_data_y = (iris["target"] == 2).astype(np.int)  # 1 if Iris-Virginica, else 0
feature_length = iris_data_X.shape[1]
data_num = iris_data_X.shape[0]
iris_data_y = iris_data_y.reshape(-1, 1)

In [40]:
iris_data_y[0:5]

array([[0],
       [0],
       [0],
       [0],
       [0]])

In [32]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_iris_data = scaler.fit_transform(iris_data_X)
scaled_iris_data_plus_bias = np.c_[np.ones((data_num, 1)), scaled_iris_data]

In [34]:
scaled_iris_data_plus_bias[0:5]

array([[ 1.        , -0.90068117,  1.03205722, -1.3412724 , -1.31297673],
       [ 1.        , -1.14301691, -0.1249576 , -1.3412724 , -1.31297673],
       [ 1.        , -1.38535265,  0.33784833, -1.39813811, -1.31297673],
       [ 1.        , -1.50652052,  0.10644536, -1.2844067 , -1.31297673],
       [ 1.        , -1.02184904,  1.26346019, -1.3412724 , -1.31297673]])

In [55]:
iris_data_y.shape

(150, 1)

In [56]:
scaled_iris_data_plus_bias.shape

(150, 5)

In [82]:
n_epochs = 20000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, feature_length + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

#one = tf.constant(1.0, shape=[data_num, 1], dtype=tf.float32, name="one")

theta = tf.Variable(tf.random_uniform([feature_length+1, 1], -1., 1.), name="theta")
h = 1 / (1 + tf.exp(-tf.matmul(X, theta, name="h")))

with tf.name_scope("loss") as scope:
    loss = - tf.matmul(tf.transpose(y), tf.log(h + 1e-10)) - tf.matmul(tf.transpose(1 - y), tf.log(1 - h + 1e-10))
    loss_avg = loss / data_num

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss_avg)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    '''
    loss_eval, loss_avg_eval = sess.run([loss, loss_avg], feed_dict={X: scaled_iris_data_plus_bias, y: iris_data_y})
    #result = log.eval(feed_dict={X: scaled_iris_data_plus_bias, y: iris_data_y})
    #result = log.eval(feed_dict={X: scaled_iris_data_plus_bias, y: iris_data_y})
    print(loss_eval, loss_avg_eval) 
    '''
    
    for epoch in range(n_epochs):
        if epoch % 1000 == 0:
            print("Epoch = ", epoch, "loss_avg = ", loss_avg.eval(feed_dict={X: scaled_iris_data_plus_bias, y: iris_data_y}))
        sess.run(training_op, feed_dict={X: scaled_iris_data_plus_bias, y: iris_data_y})  #the same as training_op.eval()
        
    best_theta = theta.eval()
    print(best_theta)

Epoch =  0 loss_avg =  [[ 1.37738609]]


Epoch =  1000 loss_avg =  [[ 0.28948277]]


Epoch =  2000 loss_avg =  [[ 0.23571312]]


Epoch =  3000 loss_avg =  [[ 0.20791094]]


Epoch =  4000 loss_avg =  [[ 0.18865919]]


...

Epoch =  16000 loss_avg =  [[ 0.1078041]]


Epoch =  17000 loss_avg =  [[ 0.10511226]]


Epoch =  18000 loss_avg =  [[ 0.10265231]]


Epoch =  19000 loss_avg =  [[ 0.10039472]]


[[-4.29850912]
 [-0.21099344]
 [-0.48030239]
 [ 2.99570107]
 [ 4.01632404]]


In [102]:
X = tf.placeholder(tf.float32, shape=(None, feature_length + 1), name="X")
h = (1 / (1 + tf.exp(-tf.matmul(X, best_theta, name="h")))) > 0.5

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    result = (h.eval(feed_dict={X: scaled_iris_data_plus_bias})).astype(np.int)
    print(result)

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


In [103]:
from sklearn.metrics import precision_score, recall_score
precision_score(iris_data_y, result)

0.94230769230769229

In [104]:
recall_score(iris_data_y, result)

0.97999999999999998